<a href="https://colab.research.google.com/github/GEORMC/Nnumerical_Methods_Course/blob/main/Newmark_Beta_SDOF_Load_Cases_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import nbformat as nbf

# Initialize parameters for Newmark-beta method
m = 1.0       # mass in kg
c = 0.1       # damping coefficient in Ns/m
k = 2.0       # stiffness in N/m
dt = 0.1      # time step in seconds
t_max = 1.0   # maximum time in seconds
gamma = 0.5   # Newmark parameter (constant average acceleration)
beta = 0.25   # Newmark parameter (constant average acceleration)

# Time steps
n_steps = int(t_max / dt) + 1
time = np.arange(0, t_max + dt, dt)

# Prepare force functions
def constant_load(t):
    return 1.0  # constant force

def variable_load(t):
    return np.sin(2 * np.pi * t)  # sinusoidal force

def earthquake_load(t):
    return 0.3 * np.exp(-((t - 0.5) ** 2) / (2 * 0.1 ** 2))  # synthetic earthquake

# Function to solve SDOF system using Newmark-beta method for a given load function
def newmark_beta(load_func):
    # Initialize displacement, velocity, and acceleration arrays
    u = np.zeros(n_steps)    # displacement
    v = np.zeros(n_steps)    # velocity
    a = np.zeros(n_steps)    # acceleration

    # Initial acceleration from equilibrium equation at t=0
    a[0] = (load_func(0) - c * v[0] - k * u[0]) / m

    # Precompute constants for Newmark method
    a0 = 1 / (beta * dt ** 2) * m + gamma / (beta * dt) * c
    a1 = 1 / (beta * dt) * m + (gamma / beta - 1) * c
    a2 = (1 / (2 * beta) - 1) * m + dt * (gamma / (2 * beta) - 1) * c

    # Iterate through each time step
    for n in range(n_steps - 1):
        # Calculate effective force
        p_eff = load_func(time[n+1]) + a0 * u[n] + a1 * v[n] + a2 * a[n]

        # Displacement at next time step
        u[n+1] = p_eff / (k + a0)

        # Velocity and acceleration at next time step
        v[n+1] = gamma / (beta * dt) * (u[n+1] - u[n]) + (1 - gamma / beta) * v[n] + dt * (1 - gamma / (2 * beta)) * a[n]
        a[n+1] = 1 / (beta * dt ** 2) * (u[n+1] - u[n]) - 1 / (beta * dt) * v[n] - (1 / (2 * beta) - 1) * a[n]

    return pd.DataFrame({"Time (s)": time, "Displacement (m)": u, "Velocity (m/s)": v, "Acceleration (m/s^2)": a})

# Calculate results for each load case
df_constant = newmark_beta(constant_load)
df_variable = newmark_beta(variable_load)
df_earthquake = newmark_beta(earthquake_load)

